# Part3: Simple Graphical Interfaces

Scipp itself does not provide a graphical user interface (GUI) apart from widgets that come with the plots.
However, the (experimental) companion package [scippwidgets](https://scipp.github.io/scippwidgets/) provides a toolkit for assembling simple GUIs in Jupyter notebooks from widgets.
Details of this package will definitely change in the future, but this serves as an example of how simple and flexible GUIs could be created.

We create some 2-D dummy data to work with.
This could represent, e.g., the counts on a 2-D image sensor:

In [ ]:
import scipp as sc
import numpy as np
a = np.random.rand(100,100)
a[23:63,16:74] += 1.0
a[20:66,13:76] += 1.0
a[19:67,12:77] += 1.0
x = sc.array(dims=['x'], unit=sc.units.m, values=np.linspace(-0.3,0.3,num=101))
y = sc.array(dims=['y'], unit=sc.units.m, values=np.linspace(-0.2,0.2,num=101))
var = sc.array(dims=['x','y'], unit=sc.units.counts, values=a, variances=a)
data = sc.DataArray(data=var, coords={'x':x, 'y':y})
data

Next we import `scippwidgets` and setup a `DisplayWidget` for creating a plot side-by-side with a line profile.
Here we are hiding the code from the user, click the "Py" button to show the cell's code:

In [ ]:
import scippwidgets as sw

def nice_label(data, dim, center):
    unit = data.coords[dim].unit
    unit_str = '\\mathrm{' + str(unit) + '}'
    label = round(data.coords[dim][dim, center].value, 3)
    return f'${label}~{unit_str}$'

def plot_profile(data=None, dim=None, center=None):
    if data is None:
        print('Missing input variable name for `data`.')
        return 
    # 1. Create 2D plot of data
    from matplotlib import pyplot as plt
    figs, axs = plt.subplots(1, 2, figsize=(8, 3))
    figs.subplots_adjust(bottom=0.2)
    out = data.plot(ax=axs[0])
    if dim is None or center is None:
        print('Enter values for `dim` and `center` to enable profile plot.')
        return figs.canvas
    # 2. Plot one (or multiple) profiles, we get a tuple if `center` has multiple comma-separated values
    if not hasattr(center, '__iter__'):
        center = (center,)
    profiles = {}
    for c in center:
        profiles[nice_label(data, dim, c)] = data[dim, c]
    sc.plot.plot(profiles, ax=axs[1])
    return figs.canvas
    
data_input = sw.inputs.ScippInputWithDim(
    func_arg_names=('data', 'dim'),
    layout={'width':'max-content'})
center_input = sw.inputs.Input(
    function_arg_name='center',
    layout={'width':'max-content'})
sw.DisplayWidget(wrapped_func=plot_profile,
                 inputs=[data_input, center_input],
                 button_name='Update',
                 hide_code=True) # Set to `True` to hide code from this cell

## Exercises

1. Modify the example to include an additional input field for `width`.
   The profile should then be computed as the sum along `dim` over a slice of the given width.
2. Modify the example so the entered values for `center` and `width` are interpreted as coordinate values rather than integer indices.
   Hint: Use [Label-based indexing](https://scipp.github.io/user-guide/slicing.html#Label-based-indexing) to extract the slice from the 2-D data.
3. Bonus: Working with `matplotlib` directly, indicate the selected profile region(s) in the 2-D plot on the left.
   Hints:
   - `axs[0]` gives access to the axes of the left (2-D) plot.
   - Use, e.g., [axhline](https://matplotlib.org/3.2.2/api/_as_gen/matplotlib.pyplot.axhline.html) and [axvline](https://matplotlib.org/3.2.2/api/_as_gen/matplotlib.pyplot.axvline.html)  or [axhspan](https://matplotlib.org/3.2.2/api/_as_gen/matplotlib.axes.Axes.axhspan.html) and [axvspan](https://matplotlib.org/3.2.2/api/_as_gen/matplotlib.pyplot.axvspan.html)

## Solution

Click the "Py" button to show the code of the solution.

In [ ]:
import scippwidgets as sw

def profile_plot():
    def make_line(axs, data, dim, loc, color, linestyle='-'):
        color = f'C{color}'
        if dim == data.dims[0]:
            axs[0].axhline(loc, color=color, linestyle=linestyle)
        else:
            axs[0].axvline(loc, color=color, linestyle=linestyle)

    def mark(axs, data, dim, start, stop):
        make_line(axs, data, dim, (stop.value+start.value)/2, mark.color)
        make_line(axs, data, dim, stop.value, mark.color, linestyle='--')
        make_line(axs, data, dim, start.value, mark.color, linestyle='--')
        mark.color +=1

    def nice_label(data, dim, center, width):
        unit = data.coords[dim].unit
        unit_str = '\\mathrm{' + str(unit) + '}'
        return f'${center:.3f}\pm{width/2}~{unit_str}$'

    def plot_profile(data=None, dim=None, center=None, width=None):
        if data is None:
            print('Missing input variable name for `data`.')
            return 
        # 1. Create 2D plot of data
        from matplotlib import pyplot as plt
        figs, axs = plt.subplots(1, 2, figsize=(8, 3))
        figs.subplots_adjust(bottom=0.2)
        out = data.plot(ax=axs[0])
        if dim is None or center is None or width is None:
            print('Enter values for `dim`, `center`, and `width` to enable profile plot.')
            return figs.canvas
        # 2. Plot one (or multiple) profiles, we get a tuple if `center` has multiple comma-separated values
        if not hasattr(center, '__iter__'):
            center = (center,)
        profiles = {}
        unit = data.coords[dim].unit
        mark.color = 0
        for c in center:
            # For exercise 1: Use start and stop based on width
            # For exercise 2: Multiply by unit to create scalar variable => label based indexing
            start = (c - width/2) * unit
            stop = (c + width/2) * unit
            profiles[nice_label(data, dim, c, width)] = sc.sum(data[dim, start:stop], dim)
            # For exercise 3: Mark profile region in 2-D plot
            mark(axs, data, dim, start, stop)
        out = sc.plot.plot(profiles, ax=axs[1])
        return figs.canvas

    data_input = sw.inputs.ScippInputWithDim(
        func_arg_names=('data', 'dim'),
        layout={'width':'max-content'})
    center_input = sw.inputs.Input(
        function_arg_name='center',
        layout={'width':'max-content'})
    # For exercise 1: Added another input
    width_input = sw.inputs.Input(
        function_arg_name='width',
        layout={'width':'max-content'})
    return sw.DisplayWidget(wrapped_func=plot_profile,
                            inputs=[data_input, center_input, width_input],
                            button_name='Update',
                            hide_code=True) # Set to `True` to hide code from this cell

profile_plot()